# 1. 필요 모듈 및 데이터 불러오기

In [1]:
import glob
import os
import re
import tensorflow as tf
import numpy as np
import pandas as pd

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:30])

데이터 크기: 187088
Examples:
 ['When somebody loved me', 'Everything was beautiful', 'Every hour we spent together', 'Lives within my heart And when she was sad', 'I was there to dry her tears', 'And when she was happy so was I', 'When she loved me Through the summer and the fall', 'We had each other that was all', 'Just she and I together', 'Like it was meant to be And when she was lonely', 'I was there to comfort her', 'And I knew that she loved me So the years went by', 'I stayed the same', 'She began to drift away', 'I was left alone Still I waited for the day', "When she'd say", 'I will always love you Lonely and forgotten', "Never thought she'd look my way", 'She smiled at me and held me Just like she used to do', 'Like she loved me', 'When she loved me When somebody loved me', 'Everything was beautiful', 'Every hour we spent together', 'Lives within my heart When she loved me, hey Sentimentos são', 'Fáceis de mudar', 'Mesmo entre quem', 'Não vê que alguém', 'Pode ser seu par Basta u

# 2. 정제 함수 만들기

In [2]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [3]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if len(sentence) > 15: continue # 문장길이 15 이상인 가사들은 무시
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> when she d say <end>',
 '<start> f ceis de mudar <end>',
 '<start> quando ele vem <end>',
 '<start> nada o det m <end>',
 '<start> para nos trazer <end>',
 '<start> novas sensa es <end>',
 '<start> doces emo es <end>',
 '<start> sentimentos s o <end>',
 '<start> como uma can o <end>',
 '<start> como uma can o <end>']

In [4]:
total_corpus = list(corpus)
num_tokens = [len(tokens) for tokens in total_corpus]
num_tokens = np.array(num_tokens)
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))


문장길이 평균 :  24.541784702549574
문장길이 최대 :  35
문장길이 표준편차 :  3.682949641428149


### exp4에서 배운 내용을 사용하여 문장 길이 정보를 열람한다.

# 3. Data set split

In [5]:
enc_train, enc_val, dec_train, dec_val = <코드 작성>

SyntaxError: invalid syntax (<ipython-input-5-851e16a23f6d>, line 1)

# 3. Tokenization

In [6]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   95   56 ...    0    0    0]
 [   2  677 1908 ...    0    0    0]
 [   2 1910 1911 ...    0    0    0]
 ...
 [   2  153   87 ...    0    0    0]
 [   2    9  417 ...    0    0    0]
 [   2 1703    3 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7efcc822b8d0>


In [7]:
tensor[0]

array([  2,  95,  56, 115,  52,   3,   0,   0,   0,   0,   0,   0,   0],
      dtype=int32)

In [8]:
print(tensor[:3, :10])

[[   2   95   56  115   52    3    0    0    0    0]
 [   2  677 1908  881 1909    3    0    0    0    0]
 [   2 1910 1911 1912    3    0    0    0    0    0]]


In [9]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : .
7 : you
8 : oh
9 : it
10 : me


In [10]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[  2  95  56 115  52   3   0   0   0   0   0   0]
[ 95  56 115  52   3   0   0   0   0   0   0   0]


In [11]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 12), (256, 12)), types: (tf.int32, tf.int32)>

In [12]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [13]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 12, 7001), dtype=float32, numpy=
array([[[ 3.07939888e-04, -8.21189460e-05, -7.56819718e-05, ...,
         -9.13202311e-05, -1.68475308e-04, -3.34178098e-04],
        [ 7.79070135e-04,  5.46524650e-04,  1.66401092e-04, ...,
          1.21163961e-04, -1.25903753e-04, -2.97368562e-04],
        [ 1.18412043e-03,  8.51486460e-04, -6.24998449e-07, ...,
          1.10699066e-05,  2.07910896e-04, -4.05409141e-04],
        ...,
        [-8.00988055e-04,  8.93340737e-04,  9.32752329e-04, ...,
          2.18048459e-03, -1.44496944e-03, -1.21907343e-03],
        [-1.09032821e-03,  9.32538765e-04,  1.23900548e-03, ...,
          2.65084510e-03, -1.76591321e-03, -1.29556179e-03],
        [-1.34392630e-03,  1.00395922e-03,  1.50110491e-03, ...,
          3.07262922e-03, -2.01218179e-03, -1.36708352e-03]],

       [[ 3.07939888e-04, -8.21189460e-05, -7.56819718e-05, ...,
         -9.13202311e-05, -1.68475308e-04, -3.34178098e-04],
        [-2.82022193e-05, -1.93959451e-04, -2.

In [14]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


In [15]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
55/55 [==============================] - 5s 82ms/step - loss: 2.6558
Epoch 2/30
55/55 [==============================] - 5s 83ms/step - loss: 1.5859
Epoch 3/30
55/55 [==============================] - 5s 83ms/step - loss: 1.4343
Epoch 4/30
55/55 [==============================] - 5s 83ms/step - loss: 1.3860
Epoch 5/30
55/55 [==============================] - 5s 83ms/step - loss: 1.3568
Epoch 6/30
55/55 [==============================] - 5s 83ms/step - loss: 1.3331
Epoch 7/30
55/55 [==============================] - 5s 84ms/step - loss: 1.3086
Epoch 8/30
55/55 [==============================] - 5s 84ms/step - loss: 1.2820
Epoch 9/30
55/55 [==============================] - 5s 84ms/step - loss: 1.2546
Epoch 10/30
55/55 [==============================] - 5s 84ms/step - loss: 1.2263
Epoch 11/30
55/55 [==============================] - 5s 84ms/step - loss: 1.1983
Epoch 12/30
55/55 [==============================] - 5s 85ms/step - loss: 1.1723
Epoch 13/30
55/55 [==================

In [18]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [21]:
generate_text(model, tokenizer, init_sentence="<start> you")

'<start> you re the one <end> '

# 짜투리 코딩

## 작사기라고는 하는데 한문장씩 나오는게 답답하여 작사기를 만들어봤다.

In [22]:
raw_corpus

['When somebody loved me',
 'Everything was beautiful',
 'Every hour we spent together',
 'Lives within my heart And when she was sad',
 'I was there to dry her tears',
 'And when she was happy so was I',
 'When she loved me Through the summer and the fall',
 'We had each other that was all',
 'Just she and I together',
 'Like it was meant to be And when she was lonely',
 'I was there to comfort her',
 'And I knew that she loved me So the years went by',
 'I stayed the same',
 'She began to drift away',
 'I was left alone Still I waited for the day',
 "When she'd say",
 'I will always love you Lonely and forgotten',
 "Never thought she'd look my way",
 'She smiled at me and held me Just like she used to do',
 'Like she loved me',
 'When she loved me When somebody loved me',
 'Everything was beautiful',
 'Every hour we spent together',
 'Lives within my heart When she loved me, hey Sentimentos são',
 'Fáceis de mudar',
 'Mesmo entre quem',
 'Não vê que alguém',
 'Pode ser seu par Basta 